In [ ]:
import numpy as np

# Pre-Processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Neural Network
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import L1L2

import plotly.graph_objects as go

In [ ]:
dataset = np.load('../00_Data/Processed-Data/classification_dataset.npz')
X = dataset["features"]
y = dataset["labels"]

In [ ]:
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.4, shuffle=True, stratify=np.ravel(y), random_state=34)
X_test, X_val, y_test, y_val = train_test_split(X_rest, y_rest, test_size=0.5, shuffle=True, stratify=np.ravel(y_rest), random_state=34)

In [ ]:
def normalize_features(X_train:np.ndarray, X_test:np.ndarray, X_val:np.ndarray, use_standard_scaler:bool=False) -> tuple:
    if(use_standard_scaler):
        scaler = StandardScaler()
    else:
        scaler = MinMaxScaler()
    X_train_norm = np.zeros(shape=(X_train.shape), dtype='float32')
    X_test_norm = np.zeros(shape=(X_test.shape), dtype='float32')
    X_val_norm = np.zeros(shape=(X_val.shape), dtype='float32')
    for feature_col in range(X_train.shape[2]):
        X_train_norm[:][:][feature_col] = scaler.fit_transform(X_train[:][:][feature_col])
        X_test_norm[:][:][feature_col] = scaler.transform(X_test[:][:][feature_col])
        X_val_norm[:][:][feature_col] = scaler.transform(X_val[:][:][feature_col])
    return X_train_norm, X_test_norm, X_val_norm

In [ ]:
X_train_norm, X_test_norm, X_val_norm = normalize_features(X_train, X_test, X_val, True)

In [ ]:
def build_and_compile_model(train_shape:tuple, initial_lr:float=0.000001):
    inputs = Input(shape=(train_shape[1], train_shape[2]))
    bilstm1 = LSTM(64, return_sequences = True)(inputs)
    do5 = Dropout(0.5)(bilstm1)
    bilstm2 = LSTM(32)(do5)
    do6 = Dropout(0.5)(bilstm2)
    d1 = Dense(16)(do6)
    do7 = Dropout(0.5)(d1)
    d2 = Dense(8)(do7)
    outputs = Dense(1, activation='sigmoid')(d2)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    opt = tf.keras.optimizers.legacy.Adam(learning_rate=initial_lr)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'])
    return model

In [ ]:
model = build_and_compile_model(X_train_norm.shape, 0.000001)

earlystopper = EarlyStopping(patience=25, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0000001, verbose=1, cooldown=10)

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    X_train, 
    y_train, 
    epochs=100, 
    batch_size=50,
    validation_data=(X_val, y_val),
    verbose=1, 
    callbacks=[earlystopper, reduce_lr]
)

In [ ]:
fig = go.Figure(
    data = [
        go.Scatter(y=history.history['loss'], name="train"),
        go.Scatter(y=history.history['val_loss'], name="val"),
    ],
    layout = {"yaxis": {"title": "Loss [MSE]"}, "xaxis": {"title": "Epoch"}, "title": "Model Loss over Epochs"}
)

fig.show()